# Apple Music Data Project

This project focuses on a dataset which was requested from apple data privacy and includes various information in relation to my music listening. The aim of the project is to find interesting insights into my music listening habits.

Readying the workspace with 'imports' and such

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
saved_style_state = matplotlib.rcParams.copy()

Importing primary dataset into workspace

In [10]:
filepath = 'Apple Music Play Activity.csv'
mode = 'r'
with open(filepath, mode) as f:
    music_data = f.read()
music_data

<_io.TextIOWrapper name='Apple Music Play Activity.csv' mode='r' encoding='cp1252'>